Car-ing is sharing, an auto dealership company for car sales and rental, is taking their services to the next level thanks to Large Language Models (LLMs).

As their newly recruited AI and NLP developer, you've been asked to prototype a chatbot app with multiple functionalities that not only assist customers but also provide support to human agents in the company.

The solution should receive textual prompts and use a variety of pre-trained Hugging Face LLMs to respond to a series of tasks, e.g. classifying the sentiment in a car’s text review, answering a customer question, summarizing or translating text, etc.

In [1]:
# !pip install transformers
# !pip install evaluate

import pandas as pd
import os
import evaluate
import torch

from transformers import AutoTokenizer
from transformers import AutoModelForQuestionAnswering
from transformers import pipeline
from transformers import logging
logging.set_verbosity(logging.WARNING)

C:\Users\Alienware\miniconda3\envs\py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1.

Use a pre-trained LLM to classify the sentiment of the five car reviews in the car_reviews.csv dataset, and evaluate the classification accuracy and F1 score of predictions.
Store the model outputs in predicted_labels, then extract the labels and map them onto a list of {0,1} integer binary labels called predictions.
Store the calculated metrics in accuracy_result and f1_result.
-----------------------------------------------------------------------------------------------------------------------------

In [2]:
car_reviews = pd.read_csv('data/car_reviews.csv', delimiter = ';')
car_reviews

,Review,Class
0,I am very satisfied with my 2014 Nissan NV SL....,POSITIVE
1,The car is fine. It's a bit loud and not very ...,NEGATIVE
2,"My first foreign car. Love it, I would buy ano...",POSITIVE
3,I've come across numerous reviews praising the...,NEGATIVE
4,I've been dreaming of owning an SUV for quite ...,POSITIVE


In [3]:
classifier = pipeline('sentiment-analysis', model='distilbert-base-uncased')
classifier

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
labels = [1 if car_reviews.Class[i] == 'POSITIVE' else 0 for i in range(car_reviews.shape[0])]
labels

[1, 0, 1, 0, 1]

In [5]:
predicted_labels = [classifier(car_reviews.Review[i]) for i in range(car_reviews.shape[0])]
predicted_labels

[[{'label': 'LABEL_0', 'score': 0.5093142986297607}],
 [{'label': 'LABEL_0', 'score': 0.5065034627914429}],
 [{'label': 'LABEL_1', 'score': 0.5018572211265564}],
 [{'label': 'LABEL_0', 'score': 0.5169278383255005}],
 [{'label': 'LABEL_0', 'score': 0.5169855356216431}]]

In [6]:
predictions = [1 if predicted_labels[i][0]['score'] > 0.5 else 0 for i in range(len(predicted_labels))]
predictions

[1, 1, 1, 1, 1]

In [7]:
accuracy = evaluate.load("accuracy")
accuracy_dict = accuracy.compute(references=labels, predictions=predictions)
accuracy_dict

{'accuracy': 0.6}

In [8]:
f1 = evaluate.load("f1")
f1_dict = f1.compute(references=labels, predictions=predictions)
f1_dict

{'f1': 0.7499999999999999}

In [9]:
accuracy_result = accuracy_dict['accuracy']
f1_result = f1_dict['f1']
f1_result

0.7499999999999999

2.

The company is recently attracting customers from Spain. Extract and pass the first two sentences of the first review in the dataset to an English-to-Spanish translation LLM. Calculate the BLEU score to assess translation quality, using the content in reference_translations.txt as references.
Store the translated text generated by the LLM in translated_review.
Store the BLEU score metric result in bleu_score.
----------------------------------------------------------------------------------------------------------------

In [10]:
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-en-es")
translator

C:\Users\Alienware\miniconda3\envs\py310\lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [11]:
first_review = car_reviews.Review[0]
first_review

'I am very satisfied with my 2014 Nissan NV SL. I use this van for my business deliveries and personal use. Camping, road trips, etc. We dont have any children so I store most of the seats in my warehouse. I wanted the passenger van for the rear air conditioning. We drove our van from Florida to California for a Cross Country trip in 2014. We averaged about 18 mpg. We drove thru a lot of rain and It was a very comfortable and stable vehicle. The V8 Nissan Titan engine is a 500k mile engine. It has been tested many times by delivery and trucking companies. This is why Nissan gives you a 5 year or 100k mile bumper to bumper warranty. Many people are scared about driving this van because of its size. But with front and rear sonar sensors, large mirrors and the back up camera. It is easy to drive. The front and rear sensors also monitor the front and rear sides of the bumpers making it easier to park close to objects. Our Nissan NV is a Tow Monster. It pulls our 5000 pound travel trailer l

In [12]:
translations_raw = translator(first_review.split('.')[:2])
translations_raw

[{'translation_text': 'Estoy muy satisfecho con mi Nissan NV SL 2014'},
 {'translation_text': 'Uso esta camioneta para mis entregas de negocios y uso personal'}]

In [13]:
translations = [translations_raw[i]['translation_text'] for i in range(len(translations_raw))]
translations

['Estoy muy satisfecho con mi Nissan NV SL 2014',
 'Uso esta camioneta para mis entregas de negocios y uso personal']

In [14]:
# Load reference translations from file
with open("data/reference_translations.txt", 'r') as file:
    lines = file.readlines()

lines

['Estoy muy satisfecho con mi Nissan NV SL 2014. Utilizo esta camioneta para mis entregas comerciales y uso personal.\n',
 'Estoy muy satisfecho con mi Nissan NV SL 2014. Uso esta furgoneta para mis entregas comerciales y uso personal.']

In [15]:
references_sentences = [line.strip() for line in lines]
references_sentences

['Estoy muy satisfecho con mi Nissan NV SL 2014. Utilizo esta camioneta para mis entregas comerciales y uso personal.',
 'Estoy muy satisfecho con mi Nissan NV SL 2014. Uso esta furgoneta para mis entregas comerciales y uso personal.']

In [16]:
bleu = evaluate.load("bleu")
bleu_score = bleu.compute(references=references_sentences, predictions=translations)
bleu_score

{'bleu': 0.20645953673295725,
 'precisions': [0.85, 0.7222222222222222, 0.5625, 0.42857142857142855],
 'brevity_penalty': 0.33287108369807955,
 'length_ratio': 0.47619047619047616,
 'translation_length': 20,
 'reference_length': 42}

3.

The 2nd review in the dataset emphasizes brand aspects. Load an extractive QA LLM such as "deepset/minilm-uncased-squad2" to formulate the question "What did he like about the brand?" and obtain an answer.
Use question and context for the two variables containing the LLM inputs: question and context.
Store the actual text answer in answer.
--------------------------------------------------------------------------------------------------------------------------

In [17]:
# Instantiate model and tokenizer
model_ckp = "deepset/minilm-uncased-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_ckp)
model = AutoModelForQuestionAnswering.from_pretrained(model_ckp)

# Define context and question, and tokenize them
context = car_reviews.Review[1]
context

Some weights of the model checkpoint at deepset/minilm-uncased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


"The car is fine. It's a bit loud and not very powerful. On one hand, compared to its peers, the interior is well-built. The transmission failed a few years ago, and the dealer replaced it under warranty with no issues. Now, about 60k miles later, the transmission is failing again. It sounds like a truck, and the issues are well-documented. The dealer tells me it is normal, refusing to do anything to resolve the issue. After owning the car for 4 years, there are many other vehicles I would purchase over this one. Initially, I really liked what the brand is about: ride quality, reliability, etc. But I will not purchase another one. Despite these concerns, I must say, the level of comfort in the car has always been satisfactory, but not worth the rest of issues found."

In [18]:
question = "What did he like about the brand?"
inputs = tokenizer(question, context, return_tensors="pt")
inputs

{'input_ids': tensor([[  101,  2054,  2106,  2002,  2066,  2055,  1996,  4435,  1029,   102,
          1996,  2482,  2003,  2986,  1012,  2009,  1005,  1055,  1037,  2978,
          5189,  1998,  2025,  2200,  3928,  1012,  2006,  2028,  2192,  1010,
          4102,  2000,  2049, 12746,  1010,  1996,  4592,  2003,  2092,  1011,
          2328,  1012,  1996,  6726,  3478,  1037,  2261,  2086,  3283,  1010,
          1998,  1996, 11033,  2999,  2009,  2104, 10943,  2100,  2007,  2053,
          3314,  1012,  2085,  1010,  2055,  3438,  2243,  2661,  2101,  1010,
          1996,  6726,  2003,  7989,  2153,  1012,  2009,  4165,  2066,  1037,
          4744,  1010,  1998,  1996,  3314,  2024,  2092,  1011,  8832,  1012,
          1996, 11033,  4136,  2033,  2009,  2003,  3671,  1010, 11193,  2000,
          2079,  2505,  2000, 10663,  1996,  3277,  1012,  2044, 19273,  1996,
          2482,  2005,  1018,  2086,  1010,  2045,  2024,  2116,  2060,  4683,
          1045,  2052,  5309,  2058,  

In [19]:
with torch.no_grad():
    outputs = model(**inputs)

outputs # contains the most likely position probability of the start and end of the answer string (start_logit, end_logit)

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[ 2.2762, -6.2343, -6.0321, -6.0678, -6.0882, -6.0398, -6.0916, -6.3243,
         -6.6169,  2.2762, -4.1784, -5.3074, -6.0259, -5.0759, -6.3563, -4.7436,
         -6.3526, -6.4298, -5.7884, -5.7856, -5.1914, -6.4162, -4.9813, -5.8880,
         -6.1990, -6.5654, -5.0210, -6.2009, -6.6289, -6.4944, -5.6861, -6.2675,
         -6.1552, -6.4367, -6.4137, -5.1342, -5.3643, -6.2587, -5.5127, -6.2846,
         -6.5086, -6.5867, -5.4555, -5.4227, -6.3117, -6.0545, -6.1566, -6.4791,
         -6.6384, -6.8284, -6.2884, -5.4548, -5.0252, -6.0445, -6.5075, -5.9411,
         -6.0409, -6.8019, -6.3508, -5.8416, -6.7474, -6.7375, -6.0882, -6.3069,
         -5.2681, -5.4771, -6.5445, -6.5645, -6.5778, -6.5040, -5.7578, -5.9364,
         -6.3930, -6.5072, -6.7121, -6.6489, -5.7286, -6.0742, -6.1914, -5.8883,
         -6.2878, -6.8106, -6.4051, -6.0894, -6.1538, -6.4427, -6.1129, -6.5051,
         -6.7101, -6.6582, -3.8438, -4.4204, -5.9236, -5

In [20]:
answer_start_index = int(outputs.start_logits.argmax())
answer_end_index = int(outputs.end_logits.argmax())
answer_start_index, answer_end_index

(138, 141)

In [21]:
answer = tokenizer.decode(inputs['input_ids'][0][answer_start_index:answer_end_index+1])
answer

'ride quality, reliability'

4.

Summarize the last review in the dataset, into approximately 50-55 tokens long. Store it in the variable summarized_text.

------------------------------------------------------------------------------------------------------------

In [22]:
model_name = "cnicu/t5-small-booksum"
summarizer = pipeline("summarization", model=model_name)
summarizer

In [23]:
last_review = car_reviews.Review.iloc[-1]
last_review

"I've been dreaming of owning an SUV for quite a while, but I've been driving cars that were already paid for during an extended period. I ultimately made the decision to transition to a brand-new car, which, of course, involved taking on new payments. However, given that I don't drive extensively, I was inclined to avoid a substantial financial commitment. The Nissan Rogue provides me with the desired SUV experience without burdening me with an exorbitant payment; the financial arrangement is quite reasonable. Handling and styling are great; I have hauled 12 bags of mulch in the back with the seats down and could have held more. I am VERY satisfied overall. I find myself needing to exercise extra caution when making lane changes, particularly owing to the blind spots resulting from the small side windows situated towards the rear of the vehicle. To address this concern, I am actively engaged in making adjustments to my mirrors and consciously reducing the frequency of lane changes. Th

In [24]:
outputs = summarizer(last_review, max_length=54)
outputs

[{'summary_text': 'the Nissan Rogue provides me with the desired SUV experience without burdening me with an exorbitant payment; the financial arrangement is quite reasonable. I have hauled 12 bags of mulch in the back with the seats down and could have held more. I'}]

In [25]:
summarized_text = outputs[0]['summary_text']
summarized_text

'the Nissan Rogue provides me with the desired SUV experience without burdening me with an exorbitant payment; the financial arrangement is quite reasonable. I have hauled 12 bags of mulch in the back with the seats down and could have held more. I'